In [46]:
from openai import OpenAI
from g4f.client import Client
from db_utils import connect_to_db, execute_sql, load_mapping_from_csv,query_database
import  re

In [47]:
client = OpenAI(api_key = 'xxxxxxxx')

In [48]:
def generate_sql_from_nl(user_input):
    mapping_table = load_mapping_from_csv()
    print(mapping_table)
    system_prompt = f''''
        1. 你是一個資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        '''

    user_prompt = '''
        資料庫結構、表名稱、欄位名稱請參考以下內容: {mapping_table}
                '''
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input + user_prompt}
        ],
        max_tokens=150,
        temperature=0.5
    )
    sql_response = response.choices[0].message.content
    return sql_response


In [49]:
def extract_sql_query(sql_response): 
    match = re.search(r'SELECT.*', sql_response, re.IGNORECASE) 
    if match: 
        return match.group() 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [50]:
if __name__ == "__main__":
    user_input ="請幫我查詢員工表內所有資訊"   #input("請輸入內容: ") 
    # 初次調用GPT判斷是否需要function calling 
    response_1 = client.chat.completions.create( model="gpt-4o-mini", 
                                                messages=[
                                                {"role": "system", 
                                                 "content":"請判斷使用者輸入是否需要連線到資料庫查詢，若需要請回答YES即可!若不需要則正常對話"},
                                                {"role": "user", "content": user_input}])
    
    print('===== 初次調用GPT判斷 =====')
    if 'YES' in response_1.choices[0].message.content:
        mapping_table = load_mapping_from_csv()
        sql_response = generate_sql_from_nl(user_input) 
        sql_query = extract_sql_query(sql_response)
        print(f'生成的SQL指令: {sql_query}') 
        # 執行GPT生成的SQL查詢 
        result = query_database(sql_query)
        print(f'查詢結果: {result}')

    else: 
        # 一般對話處理
        print(response_1.choices[0].message.content )
         


===== 初次調用GPT判斷 =====
[
    {
        "table_name": "customer",
        "table_name_cn": "顧客",
        "column_name": "CUSTID",
        "column_name_cn": "顧客ID"
    },
    {
        "table_name": "customer",
        "table_name_cn": "顧客",
        "column_name": "NAME",
        "column_name_cn": "名稱"
    },
    {
        "table_name": "customer",
        "table_name_cn": "顧客",
        "column_name": "ADDRESS",
        "column_name_cn": "地址"
    },
    {
        "table_name": "customer",
        "table_name_cn": "顧客",
        "column_name": "CITY",
        "column_name_cn": "城市"
    },
    {
        "table_name": "customer",
        "table_name_cn": "顧客",
        "column_name": "STATE",
        "column_name_cn": "州"
    },
    {
        "table_name": "customer",
        "table_name_cn": "顧客",
        "column_name": "ZIP",
        "column_name_cn": "郵政編碼"
    },
    {
        "table_name": "customer",
        "table_name_cn": "顧客",
        "column_name": "AREA",
        "column_name_cn": 